In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
import pandas as pd

df=pd.read_csv('/Users/nathanmitch/Desktop/sundress/L5T5Crime.csv')
df.columns = df.columns.str.replace(' ', '')
LocationDescription = df["LocationDescription"].unique()

app = dash.Dash()

app.layout = html.Div([
    html.H2("Crime by Year"),
    html.Div(
        [
            dcc.Dropdown(
                id="LocationDescription",
                options=[{
                    'label': i,
                    'value': i
                } for i in LocationDescription],
                value='LocationDescription'),
        ],
        style={'width': '25%',
               'display': 'inline-block'}),
    dcc.Graph(id='funnel-graph'),
])
@app.callback(
    dash.dependencies.Output('funnel-graph', 'figure'),
    [dash.dependencies.Input('LocationDescription', 'value')])
def update_graph(LocationDescription):
    if LocationDescription == "LocationDescription":
        df_plot = df.copy()
    else:
        df_plot = df[df['LocationDescription'] == LocationDescription]

    pv = pd.pivot_table(
        df_plot,
        index=['Year'],
        columns=["PrimaryType"],
        values=['ID'],
        aggfunc=len,
        fill_value=0)

    trace1 = go.Bar(x=pv.index, y=pv[('ID', 'ASSAULT')], name='ASSAULT')
    trace2 = go.Bar(x=pv.index, y=pv[('ID', 'BATTERY')], name='BATTERY')
    trace3 = go.Bar(x=pv.index, y=pv[('ID', 'CRIMINAL DAMAGE')], name='CRIMINAL DAMAGE')
    trace4 = go.Bar(x=pv.index, y=pv[('ID', 'NARCOTICS')], name='NARCOTICS')
    trace5 = go.Bar(x=pv.index, y=pv[('ID', 'THEFT')], name='THEFT')

    return {
        'data': [trace1, trace2, trace3, trace4, trace5],
        'layout':
        go.Layout(
            title='Crime in this{}'.format(LocationDescription),
            barmode='stack')
    }
if __name__ == '__main__':
    app.run_server(debug=False)